In [57]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 

In [58]:
#reading data from CSV file
dataset = pd.read_csv('D:\ELLAssDataset\health_data.csv')
dataset  = dataset.sample(frac = 1)
X = dataset.iloc[:,:-1].values
y = dataset.iloc[:,-1:].values
datasize = X.shape[0]
X_train = X[:(datasize*7)//10,:]
y_train = y[:(datasize*7)//10,:]
X_test = X[(datasize*7)//10:,:]
y_test =  y[(datasize*7)//10:,:]
print(dataset)

     age  restbps  chol  category
24    21      113   216         0
364   44      119   235         0
254   36      116   254         0
346   47      133   254         0
581   64      142   368         1
..   ...      ...   ...       ...
482   68      141   259         1
386   42      116   224         0
434   52      160   325         1
174   23      107   252         0
263   35      111   239         0

[700 rows x 4 columns]


In [59]:
X_train_0 = X_train[y_train[:,0]==0]
X_train_1 = X_train[y_train[:,0]==1]
print(X_train_1)

[[ 64 142 368]
 [ 55 156 307]
 [ 39 120 225]
 [ 57 160 348]
 [ 66 131 345]
 [ 53 140 289]
 [ 46 149 264]
 [ 48 114 217]
 [ 52 146 220]
 [ 43 130 226]
 [ 56 157 287]
 [ 34 135 251]
 [ 47 111 202]
 [ 58 166 302]
 [ 67 153 364]
 [ 65 159 351]
 [ 67 152 351]
 [ 54 155 395]
 [ 61 140 372]
 [ 40 133 214]
 [ 61 150 332]
 [ 56 149 348]
 [ 36 133 233]
 [ 40 136 236]
 [ 62 145 384]
 [ 44 134 238]
 [ 58 143 366]
 [ 71 140 368]
 [ 48 146 289]
 [ 57 141 312]
 [ 51 144 283]
 [ 52 159 290]
 [ 69 137 311]
 [ 75 131 339]
 [ 50 135 315]
 [ 72 137 310]
 [ 71 136 395]
 [ 53 165 306]
 [ 44 147 233]
 [ 68 156 378]
 [ 50 123 322]
 [ 45 137 214]
 [ 42 135 212]
 [ 50 129 289]
 [ 66 142 329]
 [ 62 146 370]
 [ 56 151 272]
 [ 51 127 330]
 [ 38 136 232]
 [ 71 138 388]
 [ 43 128 213]
 [ 45 144 309]
 [ 62 147 328]
 [ 45 120 218]
 [ 57 150 288]
 [ 39 119 247]
 [ 62 144 314]
 [ 55 149 249]
 [ 47 139 259]
 [ 42 135 225]
 [ 42 145 219]
 [ 54 135 254]
 [ 43 119 239]
 [ 53 152 292]
 [ 55 156 246]
 [ 54 135 241]
 [ 42 132 

In [60]:
#Multivariate gaussian distribution
def Multivariate_gaussian(inp,mean,covariance):
    det_covariance = np.linalg.det(covariance)
    front = ((2*np.pi)**3)*det_covariance
    front = pow(front, 0.5)
    front = 1/front
    exp = np.matmul(np.transpose(inp - mean),covariance)
    exp = np.matmul(exp,inp-mean)
    exp = np.exp(-0.5*exp)
    ret = front*exp
    return ret

In [61]:
#Calculating the mean and Variance for the class 0
N_0 = X_train_0.shape[0]
mean_0 = (1/N_0)*np.sum(X_train_0, axis = 0)
print(mean_0)
covariance_0 = (1/N_0)*(np.matmul(np.transpose(X_train_0-mean_0),X_train_0 - mean_0))
print(covariance_0)

[ 32.16071429 114.06785714 229.90714286]
[[ 88.04917092  96.36409439  -5.69221939]
 [ 96.36409439 166.41325255 -17.99369898]
 [ -5.69221939 -17.99369898 924.84137755]]


In [62]:
#Calculating the mean and covariance matrix for the class 1
N_1 = X_train_1.shape[0]
mean_1 = (1/N_1)*np.sum(X_train_1, axis = 0)
covariance_1 = (1/N_1)*(np.matmul(np.transpose(X_train_1-mean_1),X_train_1 - mean_1))
print(mean_1)
print(covariance_1)

[ 54.64285714 140.41904762 289.97142857]
[[  91.65816327   37.29727891  410.9755102 ]
 [  37.29727891  148.46249433  249.4262585 ]
 [ 410.9755102   249.4262585  3481.50394558]]


In [63]:
def predict(inp):
    p_0 = Multivariate_gaussian(inp,mean_0,covariance_0)
    p_1 = Multivariate_gaussian(inp,mean_1,covariance_1)
    if p_0 > p_1:
        return 0
    else:
        return 1

In [64]:
def Accuracy_of_classifier(test_X,test_y):
    count = 0
    score = [[0,0] for i in range(2)]
    for i in range(test_X.shape[0]):
        pred_cat = predict(test_X[i])
        score[test_y[i][0]][pred_cat]+=1
        if test_y[i][0]==pred_cat:
            count+=1
    print(score)
    precision = score[1][1]/(score[0][1]+score[1][1])
    recall = score[1][1]/(score[1][0]+score[1][1])
    F1 = (2*precision*recall)/(precision+recall)
    print("Accuracy: {} %".format(count*100/len(test_X)))
    print("Precision: {}".format(precision))
    print("Recall: {}".format(recall))
    print("F1 Score: {}".format(F1))


In [65]:
Accuracy_of_classifier(X_train,y_train)

[[1, 279], [0, 210]]
Accuracy: 43.06122448979592 %
Precision: 0.4294478527607362
Recall: 1.0
F1 Score: 0.6008583690987125


In [66]:
predict([21,105,241])

1